# Egocentric Video Mapper

Before starting make sure you have an available GPU. If you are unsure about whether you have an available GPU or if you want to check which GPU you will be working with run the next cell.

In [ ]:
!nvidia-smi -L


Fill out the path to the directory of the uncompressed `Timeseries Data + Scene Video` download from Cloud.

In [ ]:
neon_timeseries_dir = "Path/To/NeonTimeSeriesFolder"

Fill out the path of the corresponding alternative egocentric view recording (please make sure that both videos have the same orientation).

In [ ]:
alternative_vid_path = "Path/To/AlternativeVideo.ext"

Define an output directory where the different output files will be saved.

In [ ]:
output_dir = "Path/To/OutputFolder"

## Select whether you want to map fixations, the whole gaze signal or both signals

In [ ]:
mapping_choice = "Fixations"  # 'Fixations', 'Gaze' or 'Both'

### Select configurations for mapping gaze

These configurations only affect the mapping of the 200Hz gaze measurements found in the gaze.csv file.

You can control how often, in seconds, the egocentric video mapper recalculates new image matches between the videos.


<div class="alert alert-block alert-info">Note: Higher time thresholds lead to a reduction in computation time, however it may decrease mapping accuracy.</div>

If you leave the value at 0 then for every gaze measurement new image matches will be calculated (slowest option but most accurate one).

In [ ]:
refresh_time_threshold_sec = 0.5

Set the `render_video` variable to True if you want to render the mapped gaze into a copy of the alternative egocentric video (at its original frame rate).

Similarly, set the `render_video_comparison` variable to True if you want to render both egocentric videos (Neon Scene and the alternative egocentric camera) side by side showing their respective gaze measurements.

In [ ]:
render_video = False
render_video_comparison = False

## Advanced configurations

Here you can choose which optic flow algorithm will be used for video synchronization,as default we use Lucas-Kanade sparse algorithm. However in quasi-static videos or in videos with very feature poor scenes we recommend using a dense optic flow method, note that using a dense optic flow method will increase the computation time. 

You can also choose which image matcher algorithm that will guide the mapping. The publicly available Efficient LOFTR model was trained on outdoor images (MegaDepth dataset), if you would like a model specialized in indoor settings (trained on ScanNet dataset) we have available the indoor model of LOFTR.

In [ ]:
optic_flow_algorithm = "Lucas-Kanade"  # "Lucas-Kanade" or "Gunnar Farneback"
image_matcher = "Efficient_LOFTR"  # "Efficient_LOFTR", "LOFTR_indoor"

## Before running the Egocentric Video Mapper, check the alternative egocentric video orientation.

Please make sure the orientation of the alternative egocentric video is the same as the Neon Scene Camera Video. This view can sometimes differ from the video view in players like VLC or QuickTime due to metadata in the video file.

In [ ]:
from pupil_labs.egocentric_video_mapper.utils import show_videos_preview

show_videos_preview(neon_timeseries_dir, alternative_vid_path)

If the orientation matches in both videos, you can jump ahead and run the Egocentric Video Mapper!

Otherwise, choose in the cell below the rotation needed for the correct visualization of the video and execute it. The cell will run a ffmpeg command to create an orientation-corrected video in the same folder as your original alternative egocentric video.

The path to the corrected alternative egocentric video will be printed. Once the orientation looks right, update the alternative video path with this new path at the beginning of the notebook and rerun the cells above this one.

**NOTE**: This command will modify the videoplayer metadata so the video is displayed in the same orientation in both the Egocentric Video Mapper and in VLC/Quick Time video players.



In [ ]:
from pupil_labs.egocentric_video_mapper.utils import execute_ffmpeg_command

rotation = "90° clockwise"  # "90° clockwise","90° counterclockwise", "180°","0°"
video_cmd, new_alternative_vid_path = execute_ffmpeg_command(
    rotation=rotation, video_path=alternative_vid_path
)

print(f"New alternative video path: {new_alternative_vid_path}")
show_videos_preview(neon_timeseries_dir, new_alternative_vid_path)

## Now let's run the egocentric video mapper!

When everything is finished, you will find the following in the specified output folder:
- `alternative_camera_gaze.csv`: The mapped gaze signal. It follows the same structure as gaze.csv, with the frequency of the gaze signal (200Hz) being preserved. This way you can easily integrate it into your existing pipelines.
- `alternative_camera_timestamps.csv`: Synchronized UTC timestamps for every alternative egocentric video frame. It follows the same structure as world_timestamps.csv
- `alternative_camera-neon_comparison.mp4`: The comparison video showing side by side the Neon Scene camera video and the alternative camera video with their respective gaze signal overlaid.
- `alternative_camera_gaze_overlay.mp4`: A copy of the alternative egocentric video with the mapped gaze overlaid.
- `neon_optic_flow.csv`: Contains the average optic flow csv for Neon Scene video.
- `alternative_optic_flow.csv`: Contains the average optic flow csv for alternative egocentric video.
- `egocentric_video_mapper_args.json`: This file contains the different parameters and configurations used to map the gaze signal

In [ ]:
import pupil_labs.egocentric_video_mapper.__main__ as main


class Args:
    def __init__(self):
        self.neon_timeseries_dir = neon_timeseries_dir
        self.alternative_vid_path = alternative_vid_path
        self.output_dir = output_dir
        self.mapping_choice = mapping_choice
        self.optic_flow_choice = optic_flow_algorithm
        self.matcher = image_matcher
        try:
            self.refresh_time_thrshld = refresh_time_threshold_sec
        except NameError:
            self.refresh_time_thrshld = None
        try:
            self.render_comparison_video = render_video_comparison
        except NameError:
            self.render_comparison_video = False
        try:
            self.render_video = render_video
        except NameError:
            self.render_video = False


args = Args()

main.main(args)